In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lrex2')\
        .master("local[*]")\
        .getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [5]:
df = spark.read.csv("Ecommerce_Customers.csv", inferSchema=True, header=True)

In [6]:
df.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [7]:
df.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [9]:
for item in df.head(1)[0]:
        print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [10]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [11]:
df.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [12]:
assembler = VectorAssembler(
        inputCols=['Avg Session Length',
        'Time on App',
        'Time on Website',
        'Length of Membership'],
        outputCol='features'
)

In [14]:
output = assembler.transform(df)

In [15]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [16]:
final_data = output.select('features', 'Yearly Amount Spent')

In [17]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [18]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [19]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                349|
|   mean|    502.00913181256|
| stddev|  82.97858626682296|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [20]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                151|
|   mean|  493.0849809716026|
| stddev|   69.9808247288413|
|    min| 256.67058229005585|
|    max|  657.0199239376519|
+-------+-------------------+



In [21]:
lr =LinearRegression(labelCol="Yearly Amount Spent", featuresCol="features")

In [22]:
lr_model = lr.fit(train_data)

In [23]:
test_results = lr_model.evaluate(test_data)

In [24]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -4.193167360661789|
| -4.070929837303709|
|  6.709109899715031|
| -6.264407926506749|
|  21.69225507303713|
| 3.0567092016161723|
| -6.266802330934979|
| 3.2489887891131843|
|-18.167384479330394|
|-2.5309972068856723|
| 17.918634459227178|
| -6.457818846160023|
|-3.1434293216909737|
|  7.399877613411206|
|-10.567551492598227|
| 11.913533799470827|
|-13.988879892229079|
| -6.747577206913206|
|  8.066760270002646|
|-18.513462318996517|
+-------------------+
only showing top 20 rows



In [25]:
test_results.rootMeanSquaredError

9.804740954219646

In [26]:
test_results.r2

0.9802394138247958

In [27]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [28]:
unlabeles_data = test_data.select('features')

In [29]:
unlabeles_data.show()

+--------------------+
|            features|
+--------------------+
|[30.8364326747734...|
|[30.8794843441274...|
|[30.9716756438877...|
|[31.0613251567161...|
|[31.2834474760581...|
|[31.3091926408918...|
|[31.5147378578019...|
|[31.5316044825729...|
|[31.5702008293202...|
|[31.5761319713222...|
|[31.6005122003032...|
|[31.7242025238451...|
|[31.8124825597242...|
|[31.8209982016720...|
|[31.8648325480987...|
|[31.9262720263601...|
|[31.9365486184489...|
|[31.9453957483445...|
|[31.9549038566348...|
|[31.9563005605233...|
+--------------------+
only showing top 20 rows



In [30]:
predictions = lr_model.transform(unlabeles_data)

In [31]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.8364326747734...| 471.6950677876514|
|[30.8794843441274...| 494.2775298221584|
|[30.9716756438877...| 487.9294998571777|
|[31.0613251567161...|493.81986598440835|
|[31.2834474760581...| 570.0888343526303|
|[31.3091926408918...|429.66400863831745|
|[31.5147378578019...| 496.0792903273964|
|[31.5316044825729...| 433.2666169402494|
|[31.5702008293202...| 564.1128766207353|
|[31.5761319713222...|  543.757581196214|
|[31.6005122003032...|461.25421703186976|
|[31.7242025238451...| 509.8457061341205|
|[31.8124825597242...| 395.9537743054882|
|[31.8209982016720...|417.27540339980214|
|[31.8648325480987...| 450.4588319694119|
|[31.9262720263601...| 380.2913996448556|
|[31.9365486184489...| 441.1882647875573|
|[31.9453957483445...| 663.7675011445651|
|[31.9549038566348...| 431.9311196699243|
|[31.9563005605233...| 565.6393940661953|
+--------------------+------------

In [32]:
spark.stop()